# Análine de dados do Thermal Shift

Vitória Yumi Uetuki Nicoleti

Observações iniciais:

Tenha certeza de que o arquivo Excel que contém os dados que quer analisar está na mesma pasta que este '.ipynb' em questão. 

**Importação das bibliotecas que são necessárias para o código**

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from lmfit.models import StepModel
import math
import seaborn as sns

**Definições das funções**

In [ ]:
def encontra_maximo(lista):
    valor = lista[0]
    for n in lista:
        if n > valor:
            valor = n
    return valor

def encontra_minimo(lista):
    valor = lista[0]
    for n in lista:
        if n < valor:
            valor = n
    return valor

def derivada_xy(x_vals, y_vals):
    avg_x_vals = []
    deriv_vals = []
    for i in range(len(x_vals) - 1):
        deriv = ( y_vals[i + 1] - y_vals[i] ) / ( x_vals[i + 1] - x_vals[i] )
        avg_x = ( x_vals[i + 1] + x_vals[i] ) / 2

        avg_x_vals.append(avg_x)
        deriv_vals.append(deriv)

    return avg_x_vals, deriv_vals 

def sigmoide(x, A, u, o):
    
    a = (x-u)/o
    f = A*(1-(1/(1 + e ** (a))))
    
    return f

def getIndexes(dfObj, value):
     
    # Empty list
    listOfPos = []
     
    # isin() method will return a dataframe with
    # boolean values, True at the positions   
    # where element exists
    result = dfObj.isin([value])
     
    # any() method will return
    # a boolean series
    seriesObj = result.any()
 
    # Get list of column names where
    # element exists
    columnNames = list(seriesObj[seriesObj == True].index)
    
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
 
        for row in rows:
            listOfPos.append((row, col))
             
    # This list contains a list tuples with
    # the index of element in the dataframe
    return listOfPos

e = np.e

def tratamento_dados_poco():
    
    """ Essa função é referente a todo o processo de tratamento de dados de um poço, desde sua escolha até o retorno de sua Tm e plot """
   
    ## Escolha de poço
    print("Qual o poço que quer plotar? (exemplo: D4)")
    POCO_PLOTAR_i = input()
    POCO_PLOTAR = pocos.index(POCO_PLOTAR_i)
    
    ## Plotagem dos seus dados
    df_poco_plotar = dfs[POCO_PLOTAR]
    df_poco_plotar.plot("Reading", "Fluorescence", linestyle="", marker="o")
    plt.title(POCO_PLOTAR_i)
    plt.show()
    
    ## Observação inicial
    print("Caso o gráfico já indique a desnaturação da proteína (não dê para modelar sigmóide, só decresce), responda 'Desnaturou'; Caso dê para modelar, responda 'Modelar'")
    OBS = input()
    Observacoes.append(OBS)
    
    while OBS.lower() not in ["desnaturou", "modelar"]:
        print("Escreva conforme o indicado")
        OBS = input()
    
    if OBS.lower() == 'desnaturou':
        Tm = 0
        Tms.append(Tm)
        resposta = print("A proteína já estava desnaturada nessas condições desde o início. Logo, atribuiu-se Tm igual a ", Tm)
          
        return resposta
    
    elif OBS.lower() == 'modelar':
    
        ## Perguntas sobre corte
        print("Considerar dados a partir de qual valor de Reading do gráfico?")
        CORTE_c = int(input())
        print("Considerar dados até qual valor de Reading do gráfico?")
        CORTE_t = int(input())

        ## Corte
        logic_c = df_poco_plotar["Reading"] >= CORTE_c
        df_poco_plotar = df_poco_plotar.loc[logic_c]
        logic_t = df_poco_plotar["Reading"] <= CORTE_t
        df_poco_plotar = df_poco_plotar.loc[logic_t]

        ## Plotagem da edição
        print("Após corte:")
        df_poco_plotar.plot("Reading", "Fluorescence", linestyle="", marker="o")
        plt.title(POCO_PLOTAR_i)
        plt.show()
        
        ## Conferindo
        print("Caso queira refazer o corte, responda 'Refazer'; Caso já esteja satisfeito(a), responda 'Prosseguir'")
        CORTE = input()
        
        while CORTE.lower() not in ["refazer", "prosseguir"]:
            print("Escreva conforme o indicado")
            CORTE = input()
            
        while CORTE.lower() == 'refazer':
            print("Considerar dados a partir de qual valor de Reading do gráfico?")
            CORTE_c = int(input())
            print("Considerar dados até qual valor de Reading do gráfico?")
            CORTE_t = int(input())
            
            df_poco_plotar = dfs[POCO_PLOTAR]
            
            ## Corte
            logic_c = df_poco_plotar["Reading"] >= CORTE_c
            df_poco_plotar = df_poco_plotar.loc[logic_c]
            logic_t = df_poco_plotar["Reading"] <= CORTE_t
            df_poco_plotar = df_poco_plotar.loc[logic_t]

            ## Plotagem da edição
            print("Após corte:")
            df_poco_plotar.plot("Reading", "Fluorescence", linestyle="", marker="o")
            plt.title(POCO_PLOTAR_i)
            plt.show()
            
            print("Caso queira refazer o corte, responda 'Refazer'; Caso já esteja satisfeito(a), responda 'Prosseguir'")
            CORTE = input()

        ## Normalização
        eixo_y = list(df_poco_plotar["Fluorescence"])
        valor_max = encontra_maximo(eixo_y)
        valor_min = encontra_minimo(eixo_y)
        Fluorescence_normalized = []
        for n in eixo_y:
            novo_valor = (n - valor_min)/(valor_max - valor_min) * 100
            Fluorescence_normalized.append(novo_valor)
        df_poco_plotar.insert(4, "Fluorescence_normalized", Fluorescence_normalized, allow_duplicates=False)
        print("Dataframe do poço ", POCO_PLOTAR_i, " com normalização:")
        display(df_poco_plotar)

        ## Plotagem da normalização
        print("Após normalização:")
        plt.scatter(df_poco_plotar["Reading"], df_poco_plotar["Fluorescence_normalized"])
        plt.title(POCO_PLOTAR_i)
        plt.show()

        ## Implementação do modelo
        modelo_sigmoid1 = StepModel(prefix="sigmoid1_", form="logistic")

        params_sigmoid1 = modelo_sigmoid1.make_params()

        params_sigmoid1.pretty_print(columns=['value', 'min', 'max', 'vary', 'expr'])

        ## Pergunta de parâmetros
        print("Observando o gráfico anterior...")
        print("Qual maior valor pra amplitude da sigmoide? (exemplo: 100)")
        A_max = int(input())
        print("Qual menor valor pra amplitude da sigmoide? (exemplo: 98)")
        A_min = int(input())
        print("Qual valor estimado você atribuiria pra amplitude da sigmoide? (exemplo: 100)")
        A_estimado = int(input())
        print("Qual maior valor para o centro da sigmoide? (exemplo: 30)")
        C_max = int(input())
        print("Qual menor valor para o centro da sigmoide? (exemplo: 20)")
        C_min = int(input())
        print("Qual valor estimado você atribuiria para o centro da sigmoide? (exemplo: 24)")
        C_estimado = int(input())

        ## Redefinição dos parâmetros
        params_sigmoid1["sigmoid1_amplitude"].set(value=A_estimado, min=A_min, max=A_max)
        params_sigmoid1["sigmoid1_center"].set(value=C_estimado, min=C_min, max=C_max)
        params_sigmoid1["sigmoid1_sigma"].set(value=10, min=0, max=100)

        ## Plotando modelo fitado      
        dados_x = df_poco_plotar["Reading"].values
        dados_y = df_poco_plotar["Fluorescence_normalized"].values

        print("Modelagem sigmóide:")

        resultado_fit = modelo_sigmoid1.fit(dados_y, params_sigmoid1, x=dados_x)
        print(resultado_fit.fit_report())

        resultado_fit.plot()
        plt.title(POCO_PLOTAR_i)
        plt.show()

        ## Recolhendo informações do modelo
        print("Copie e cole para as respostas abaixo os valores mostrados nas variáveis na modelagem")
        print("Qual o valor atribuido para a amplitude? (exemplo: 100.000000)")
        A = float(input())
        print("Qual o valor atribuido para o  centro? (exemplo: 23.1398610)")
        u = float(input())
        print("Qual o valor atribuido para o sigma? (exemplo: 2.66142358)")
        o = float(input())

        ## Estabelecendo dados obtidos com a sigmoide modelada
        valores_x = range(CORTE_c, CORTE_t+1)
        sigmoid_y = []
        for x in valores_x:
            y = sigmoide(x, A, u, o)
            sigmoid_y.append(y)
        ## Buscando o ponto de inflexão
        medias, derivadas = derivada_xy(valores_x, sigmoid_y)
        p_i = encontra_maximo(derivadas)
        index_p_i = derivadas.index(p_i)
        print("O ponto de inflexão é na máxima derivada igual a ", p_i)
        print("A leitura correspondente a essa derivada é a ", valores_x[index_p_i])
        print("Sua posição no dataframe do poço é ", index_p_i)

        ## Relacionando à Tm
        Tm_i_c = getIndexes(df_poco_plotar, valores_x[index_p_i])
        Tm_i = Tm_i_c[0][0]
        Tm = df_poco_plotar["Temperature"][Tm_i]
        Tms.append(Tm)

        resposta = print("A temperatura de 'melting' é ", Tm)

        return resposta 

**Parte inicial - Definindo base para a análise**

(Só é preciso rodar no começo)

In [ ]:
print("Coloque o nome do arquivo da forma que estiver salvo no computador: (exemplo: arquivo.xls)")
NOME_ARQUIVO = input()
print("Coloque o nome da planilha que quer usar: (exemplo: Melt Curve Raw Data)")
NOME_DA_PLANILHA = input()
print("Coloque, se tiver, em que linha que o cabeçalho termina, sendo que a 1ª é considerada 0: (exemplo: no caso em que o cabeçalho termina na linha 35 da planilha, a resposta deve ser: 34)")
LINHA_CABECALHO = int(input())
print("Coloque quais colunas da tabela contém os dados que quer usar, sendo a 1ª considerada como coluna 0: (exemplo: quer se usar da 2ª até a 5ª coluna, a resposta esperada é: 1, 2, 3, 4)")
COLUNAS_CONTENDO_OS_DADOS_i = input().split(',')
COLUNAS_CONTENDO_OS_DADOS = []
for n in COLUNAS_CONTENDO_OS_DADOS_i:
    COLUNAS_CONTENDO_OS_DADOS.append(int(n))
    
####

df = pd.read_excel(
    NOME_ARQUIVO,
    sheet_name=NOME_DA_PLANILHA,
    header=LINHA_CABECALHO,
    usecols=COLUNAS_CONTENDO_OS_DADOS,
)

df = df.dropna()

####

df.rename(columns={'Well Position': 'Well_Position'}, inplace = True)

####

print("Dataframe de todos os dados a serem usados:")
display(df)

####

groups = df.groupby(df.Well_Position)

####

pocos_r = list(df.Well_Position)
pocos = list(dict.fromkeys(pocos_r))
dfs =[]
for p in pocos:
    poco_df = groups.get_group(p)
    dfs.append(poco_df)
    
####

#dfs

####

Tms = []

Observacoes = []

**Tratamento de dados do poço**

* Repita a função:

    tratamento_dados_poco()

    para todo poco que quiser realizar o tratamento de dados. (Não precisa dar argumentos)

* Priorize realizar em ordem, pois os dados serão armazenados na sequência que rodar a função, para montar um heat map.

* É uma análise simples, basta ir respondendo às perguntas que forem sendo mostradas. No decorrer, serão mostrados os Dataframes formados, os gráficos que forem sendo obtidos, a modelagem, variáveis utilizadas e valores obtidos.

In [ ]:
tratamento_dados_poco()

**Heatmap**

É só rodar

In [ ]:
df_heatmap = pd.DataFrame(pocos)
df_heatmap.insert(1, "Tms", Tms, allow_duplicates=False)

colunas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
linhas = ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D','D', 'E', 'E', 'E', 'E','E', 'E', 'E', 'E','E','E','E', 'E', 'F', 'F', 'F','F', 'F','F', 'F','F','F','F','F','F', 'G', 'G','G','G','G','G','G','G','G','G','G','G', 'H', 'H', 'H','H', 'H', 'H', 'H', 'H','H','H','H','H']
df_heatmap.insert(1, "Colunas", colunas, allow_duplicates=False)
df_heatmap.insert(1, "Linhas", linhas, allow_duplicates=False)

del df_heatmap[0]

df_heatmap

In [ ]:
df_heatmap = df_heatmap.pivot("Linhas", "Colunas", "Tms")
df_heatmap

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.heatmap(df_heatmap, cmap="seismic", annot=True, fmt='.2f')

Observar o menor valor e alterar o *vmin* abaixo; fazer o mesmo com o maior valor e alterar o *vmax*

In [ ]:
print("Qual o menor valor, exceto 0, que é possível observar? (exemplo: 33.91)")
VMIN = float(input())
print("Qual o maior valor que é possível observar? (exemplo: 44.83)")
VMAX = float(input())
print("Quantas casas depois da vírgula quer que apareça? (use da seguinte forma: quer que apareça 2 casas, resposta: .2f)")
FMT = input()

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.heatmap(df_heatmap, vmin = VMIN, vmax = VMAX, cmap="seismic", annot=True, fmt=FMT)